<a href="https://colab.research.google.com/github/NadellaTarun/Sustainable-Federated-Learning-system-of-ocean-health-monitorings/blob/main/Final_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit pyngrok pandas numpy scikit-learn matplotlib


In [ ]:
from pyngrok import ngrok

# Add your ngrok token
!ngrok config add-authtoken 2uTvgEzz6DWGNMbfVMgoWetZPz0_7uwMsVByuiXK3D9nEK1uQ

# Kill previous tunnels (if any)
ngrok.kill()


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
%%writefile monitoring.py
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from datetime import datetime

# --------------------------
# 1. Load & Prepare Historical Data
# --------------------------
DATA_FILE = "water.csv"
df = pd.read_csv(DATA_FILE)

# Convert numeric columns
num_cols = ['Lat_Deg', 'Depthm', 'T_degC', 'PO4uM', 'SiO3uM', 'NO2uM', 'NO3uM', 'Salnty', 'O2ml_L']
for col in num_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(df[col].median())

# Parse dates
if 'Date' in df.columns:
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Create Longitude if missing
if 'Longitude' not in df.columns:
    def create_longitude(row):
        zone = str(row.get('Zone', 'Unknown'))
        station_id = str(row.get('Sta_ID', ''))
        zone_hash = hash(zone) % 360 - 180
        station_hash = hash(station_id) % 60 - 30
        base_lon = zone_hash + (station_hash * 0.1)
        return np.clip(base_lon, -180, 180)
    df['Longitude'] = df.apply(create_longitude, axis=1)

# Create Location ID
def create_location_id(row):
    lat = row['Lat_Deg']
    lon = row['Longitude']
    zone = str(row.get('Zone', 'Unknown'))
    if pd.isna(lat):
        return f"Zone_{zone}_Unknown"
    lat_bucket = int(lat // 2) * 2
    lon_bucket = int(lon // 2) * 2
    return f"Zone_{zone}_Lat{lat_bucket}_Lon{lon_bucket}"

df['Location'] = df.apply(create_location_id, axis=1)

# --------------------------
# 2. Dynamic Safe Thresholds Per Location
# --------------------------
def compute_thresholds_for_all():
    thresholds_map = {}
    for loc, data in df.groupby('Location'):
        thresholds_map[loc] = {
            'temp': (data['T_degC'].quantile(0.05), data['T_degC'].quantile(0.95)),
            'sal': (data['Salnty'].quantile(0.05), data['Salnty'].quantile(0.95)),
            'o2': (data['O2ml_L'].quantile(0.05), data['O2ml_L'].quantile(0.95))
        }
    return thresholds_map

thresholds_map = compute_thresholds_for_all()

# --------------------------
# 3. Simulation Engine (Uses Historical Data)
# --------------------------
class OceanographicDataGenerator:
    def __init__(self, location_id, historical_data):
        self.location_id = location_id
        self.historical_data = historical_data
        self.parameters = ['T_degC', 'Salnty', 'O2ml_L']

    def generate_realtime_reading(self):
        reading = {'Location': self.location_id, 'timestamp': datetime.now()}
        for p in self.parameters:
            reading[p] = self.historical_data[p].sample(1).iloc[0]
        return reading

# Create generators dynamically
generators = {
    region: OceanographicDataGenerator(region, data)
    for region, data in df.groupby('Location') if len(data) >= 10
}

# --------------------------
# 4. Train Local Models (Dynamic)
# --------------------------
def train_local_models():
    region_models = {}
    for region, region_data in df.groupby('Location'):
        if len(region_data) < 20:
            continue

        # Label dynamically based on thresholds for that region
        bounds = thresholds_map[region]
        temp_bounds = bounds['temp']
        sal_bounds = bounds['sal']
        o2_bounds = bounds['o2']

        region_data = region_data.copy()
        region_data['Label'] = region_data.apply(
            lambda row: 1 if (row['T_degC'] < temp_bounds[0] or row['T_degC'] > temp_bounds[1] or
                              row['Salnty'] < sal_bounds[0] or row['Salnty'] > sal_bounds[1] or
                              row['O2ml_L'] < o2_bounds[0] or row['O2ml_L'] > o2_bounds[1]) else 0,
            axis=1
        )

        X_train, _, y_train, _ = train_test_split(
            region_data[['T_degC', 'Salnty', 'O2ml_L']],
            region_data['Label'],
            test_size=0.2,
            random_state=42
        )

        model = RandomForestClassifier(n_estimators=50, random_state=42)
        model.fit(X_train, y_train)
        region_models[region] = model

    return region_models

region_models = train_local_models()

# --------------------------
# 5. Federated Prediction (Dynamic)
# --------------------------
def federated_predict(reading):
    features_df = pd.DataFrame([{
        'T_degC': reading['T_degC'],
        'Salnty': reading['Salnty'],
        'O2ml_L': reading['O2ml_L']
    }])

    probs = []
    for model in region_models.values():
        model_proba = model.predict_proba(features_df)[0]
        # Handle single-class models
        if len(model.classes_) == 1:
            prob_danger = 1.0 if model.classes_[0] == 1 else 0.0
        else:
            prob_danger = model_proba[list(model.classes_).index(1)]
        probs.append(prob_danger)

    avg_prob = np.mean(probs) if probs else 0.0
    dynamic_cutoff = np.mean(probs) + 0.1  # Adaptive
    return 1 if avg_prob > dynamic_cutoff else 0

# --------------------------
# 6. Threshold Checks (Per Location)
# --------------------------
def check_thresholds(reading):
    loc = reading['Location']
    bounds = thresholds_map.get(loc)

    alerts = []
    if bounds:
        temp_bounds = bounds['temp']
        sal_bounds = bounds['sal']
        o2_bounds = bounds['o2']

        if not (temp_bounds[0] <= reading['T_degC'] <= temp_bounds[1]):
            alerts.append(f"Temperature out of range ({reading['T_degC']:.2f})")
        if not (sal_bounds[0] <= reading['Salnty'] <= sal_bounds[1]):
            alerts.append(f"Salinity out of range ({reading['Salnty']:.2f})")
        if not (o2_bounds[0] <= reading['O2ml_L'] <= o2_bounds[1]):
            alerts.append(f"Oxygen out of range ({reading['O2ml_L']:.2f})")

    return alerts


Overwriting monitoring.py


In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import time
import numpy as np
import io
import soundfile as sf
from monitoring import generators, federated_predict, check_thresholds, thresholds_map
import base64

# -------------------------
# Background image
# -------------------------
def set_bg_image(image_path):
    with open(image_path, "rb") as f:
        img_bytes = f.read()
    encoded = base64.b64encode(img_bytes).decode()
    page_bg_img = f"""
    <style>
    .stApp {{
        background-image: url("data:image/png;base64,{encoded}");
        background-size: cover;
        background-position: center;
        background-attachment: fixed;
    }}
    .css-18e3th9 {{
        background: rgba(255, 255, 255, 0.8);
        border-radius: 10px;
        padding: 10px;
    }}
    </style>
    """
    st.markdown(page_bg_img, unsafe_allow_html=True)

set_bg_image("ocean.jpg")

# -------------------------
# Generate beep sound (autoplay)
# -------------------------
def play_beep():
    sr = 44100
    t = np.linspace(0, 0.5, int(sr * 0.5), endpoint=False)
    beep = 0.5 * np.sin(2 * np.pi * 440 * t)
    buf = io.BytesIO()
    sf.write(buf, beep, sr, format='WAV')

    # Encode to base64 and autoplay
    audio_base64 = base64.b64encode(buf.getvalue()).decode()
    audio_html = f"""
    <audio autoplay>
        <source src="data:audio/wav;base64,{audio_base64}" type="audio/wav">
    </audio>
    """
    st.markdown(audio_html, unsafe_allow_html=True)

# -------------------------
# Session state
# -------------------------
if "monitoring" not in st.session_state:
    st.session_state.monitoring = False
if "live_readings" not in st.session_state:
    st.session_state.live_readings = []

# -------------------------
# UI Setup
# -------------------------
st.set_page_config(page_title="Ocean Monitoring Dashboard", layout="wide")
st.markdown("<h1 style='text-align:center;color:#000000;'>🌊 Ocean Digital Twin Monitoring</h1>", unsafe_allow_html=True)

# -------------------------
# Show Dynamic Thresholds in Sidebar (Per Location)
# -------------------------
st.sidebar.subheader("Dynamic Safe Thresholds Per Location")
for loc, bounds in thresholds_map.items():
    st.sidebar.markdown(f"**{loc}**")
    st.sidebar.markdown(f"- Temp (°C): {bounds['temp'][0]:.2f} – {bounds['temp'][1]:.2f}")
    st.sidebar.markdown(f"- Salinity (PSU): {bounds['sal'][0]:.2f} – {bounds['sal'][1]:.2f}")
    st.sidebar.markdown(f"- Oxygen (ml/L): {bounds['o2'][0]:.2f} – {bounds['o2'][1]:.2f}")
    st.sidebar.markdown("---")

# Sidebar controls
refresh_rate = 1  # Fixed refresh interval
show_alerts = st.sidebar.checkbox("Show Alerts Only", value=False)

# Location filter
available_locations = list(generators.keys())
selected_locations = st.sidebar.multiselect(
    "Select Locations",
    options=available_locations,
    default=available_locations
)

# Start/Stop buttons
col1, col2 = st.columns(2)
with col1:
    if st.button("▶️ Start Monitoring"):
        st.session_state.monitoring = True
        st.session_state.live_readings = []  # clear old
with col2:
    if st.button("⏹ Stop Monitoring"):
        st.session_state.monitoring = False

# Placeholders
status_placeholder = st.empty()
placeholder = st.empty()

# -------------------------
# Monitoring Loop
# -------------------------
while st.session_state.monitoring:
    # Generate readings for selected locations only
    for loc in selected_locations:
        if loc not in generators:
            continue

        gen = generators[loc]
        reading = gen.generate_realtime_reading()
        alerts = check_thresholds(reading)
        ml_pred = federated_predict(reading)
        if ml_pred == 1 and not alerts:
            alerts.append("ML model flagged danger")

        reading['Status'] = "⚠️ DANGER" if alerts else "✅ SAFE"
        st.session_state.live_readings.append(reading)

        # Show status
        status_color = "red" if reading['Status'] == "⚠️ DANGER" else "green"
        status_placeholder.markdown(
            f"<h2 style='color:{status_color};'>{reading['Status']} (Location: {loc})<br>"
            f"Temperature: {reading['T_degC']:.2f} °C | "
            f"Salinity: {reading['Salnty']:.2f} PSU | "
            f"Oxygen: {reading['O2ml_L']:.2f} ml/L</h2>",
            unsafe_allow_html=True
        )

        if reading['Status'] == "⚠️ DANGER":
            play_beep()

    # Convert to DataFrame
    df_live = pd.DataFrame(st.session_state.live_readings)

    # Filter for display
    if show_alerts:
        df_display = df_live[df_live['Status'] == "⚠️ DANGER"]
    else:
        df_display = df_live
    if selected_locations:
        df_display = df_display[df_display['Location'].isin(selected_locations)]

    # Update table + plots
    with placeholder.container():
        st.subheader("Live Data (Latest 20 readings)")
        st.dataframe(df_display.tail(20))

        fig, ax = plt.subplots(3, 1, figsize=(10, 10))

        if not df_display.empty:
            # Use thresholds of first location in display
            first_loc = df_display.iloc[-1]['Location']
            bounds = thresholds_map.get(first_loc)

            # Temperature
            df_display.plot(x='timestamp', y='T_degC', ax=ax[0], color='blue', title="Temperature")
            ax[0].axhline(bounds['temp'][1], color='red', linestyle='--')
            ax[0].axhline(bounds['temp'][0], color='blue', linestyle='--')

            # Salinity
            df_display.plot(x='timestamp', y='Salnty', ax=ax[1], color='green', title="Salinity")
            ax[1].axhline(bounds['sal'][1], color='red', linestyle='--')
            ax[1].axhline(bounds['sal'][0], color='blue', linestyle='--')

            # Oxygen
            df_display.plot(x='timestamp', y='O2ml_L', ax=ax[2], color='orange', title="Oxygen")
            ax[2].axhline(bounds['o2'][1], color='red', linestyle='--')
            ax[2].axhline(bounds['o2'][0], color='blue', linestyle='--')

            # Rotate timestamps
            for axis in ax:
                axis.tick_params(axis='x', rotation=45)
                axis.set_xlabel("")

        plt.tight_layout(pad=5.0)
        st.pyplot(fig)

        # Extra space
        st.markdown("<div style='margin-bottom: 30px;'></div>", unsafe_allow_html=True)

    time.sleep(refresh_rate)

# -------------------------
# Download CSV
# -------------------------
if not st.session_state.monitoring and st.session_state.live_readings:
    df_download = pd.DataFrame(st.session_state.live_readings)
    st.subheader("Download Recorded Data")
    csv = df_download.to_csv(index=False).encode('utf-8')
    st.download_button(
        label="Download as CSV",
        data=csv,
        file_name="ocean_live_readings.csv",
        mime="text/csv"
    )

# -------------------------
# Upload CSV and Show Graph (Small button style)
# -------------------------
st.markdown("""
    <style>
    div[data-testid="stFileUploader"] > label > div {
        padding: 2px 10px;
        font-size: 14px;
        border: 1px solid #d3d3d3;
        border-radius: 5px;
        background-color: white;
        color: black;
        display: inline-block;
        cursor: pointer;
    }
    </style>
""", unsafe_allow_html=True)

uploaded_file = st.file_uploader(
    "Upload a CSV to analyze previous readings",
    type=["csv"],
    key="small_upload",
    label_visibility="visible"
)

if uploaded_file is not None:
    uploaded_df = pd.read_csv(uploaded_file)

    st.subheader("Uploaded Data Preview")
    st.dataframe(uploaded_df.head())

    # --- Recalculate thresholds for uploaded data ---
    def compute_safe_thresholds(df, parameter):
        return df[parameter].quantile(0.05), df[parameter].quantile(0.95)

    uploaded_temp_bounds = compute_safe_thresholds(uploaded_df, 'T_degC')
    uploaded_sal_bounds = compute_safe_thresholds(uploaded_df, 'Salnty')
    uploaded_o2_bounds = compute_safe_thresholds(uploaded_df, 'O2ml_L')

    # Plot graphs for uploaded data
    fig, ax = plt.subplots(3, 1, figsize=(10, 10))

    if not uploaded_df.empty:
        # Temperature
        uploaded_df.plot(x='timestamp', y='T_degC', ax=ax[0], color='blue', title="Temperature (Uploaded Data)")
        ax[0].axhline(uploaded_temp_bounds[1], color='red', linestyle='--')
        ax[0].axhline(uploaded_temp_bounds[0], color='blue', linestyle='--')

        # Salinity
        uploaded_df.plot(x='timestamp', y='Salnty', ax=ax[1], color='green', title="Salinity (Uploaded Data)")
        ax[1].axhline(uploaded_sal_bounds[1], color='red', linestyle='--')
        ax[1].axhline(uploaded_sal_bounds[0], color='blue', linestyle='--')

        # Oxygen
        uploaded_df.plot(x='timestamp', y='O2ml_L', ax=ax[2], color='orange', title="Oxygen (Uploaded Data)")
        ax[2].axhline(uploaded_o2_bounds[1], color='red', linestyle='--')
        ax[2].axhline(uploaded_o2_bounds[0], color='blue', linestyle='--')

        for axis in ax:
            axis.tick_params(axis='x', rotation=45)
            axis.set_xlabel("")

    plt.tight_layout(pad=5.0)
    st.pyplot(fig)

    st.markdown("<div style='margin-bottom: 30px;'></div>", unsafe_allow_html=True)

else:
    st.markdown("<div style='margin-top: 20px;'></div>", unsafe_allow_html=True)
    st.info("Upload a CSV file to view graphs.")


Writing app.py


In [ ]:
from pyngrok import ngrok
ngrok.kill()


In [ ]:
from pyngrok import ngrok
import subprocess

# Start Streamlit
print("Starting Streamlit...")
subprocess.Popen(["streamlit", "run", "app.py", "--server.port", "8501"])

# Public URL
public_url = ngrok.connect(8501)
print("Your app is live at:", public_url)


Starting Streamlit...
Your app is live at: NgrokTunnel: "https://fccf8827a2cf.ngrok-free.app" -> "http://localhost:8501"
